In [15]:
import pandas as pd

import itertools
import torch
import torch.nn as nn

from torch import tensor
from torch.utils.data import Dataset


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
device

'mps'

In [16]:
def process(multi_series):
  multi_series["timestamp"] = pd.to_datetime(multi_series["timestamp"],utc=True)
  multi_series["hour"] = multi_series["timestamp"].dt.hour
  multi_series["anglez_times_enmo"] = multi_series["anglez"] * multi_series["enmo"]
  multi_series["anglez_max"] = multi_series.groupby("series_id")["anglez"].transform('max')
  multi_series["anglez_min"] = multi_series.groupby("series_id")["anglez"].transform('min')
  multi_series["enmo_max"] = multi_series.groupby("series_id")["enmo"].transform('max')
  return multi_series



In [17]:
series_ids = ['08db4255286f', '0a96f4993bd7', '0cfc06c129cc', '1087d7b0ff2e', '10f8bc1f7b07', '18b61dd5aae8', '29c75c018220', '31011ade7c0a']
multi_series = pd.read_parquet("Zzzs_train.parquet", filters=[('series_id','in',series_ids)])


multi_series = process(multi_series)
multi_series.head()

multi_series.shape

(2844900, 11)

In [18]:
scaler = StandardScaler()
df_train_X_scaled = scaler.fit_transform(multi_series[["step", "hour","anglez","enmo","anglez_times_enmo", "anglez_max", "anglez_min", "enmo_max"]])

df_train_y = multi_series["awake"]

/Users/chenhanlyu/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/chenhanlyu/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/chenhanlyu/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/chenhanlyu/miniconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future ver

In [19]:
Xtr, Xva, ytr, yva = train_test_split(df_train_X_scaled, df_train_y, shuffle = False)

Xtr = tensor(Xtr, dtype=torch.float32).to(device)
Xva = tensor(Xva, dtype=torch.float32).to(device)
# print(ytr, yva)
ytr = tensor(ytr.values, dtype=torch.float32).to(device)
yva = tensor(yva.values, dtype=torch.float32).to(device)
Xtr.shape, Xva.shape, ytr.shape, yva.shape

class MyDataset(Dataset):
    def __init__(self, X, y):
        super(MyDataset, self).__init__()
        self.X, self.y = X, y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_dataset = MyDataset(Xtr, ytr)
valid_dataset = MyDataset(Xva, yva)

batch_size = 288  #tried 50, 100, 300

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [20]:
NUM_EPOCHS = 20 
LEARNING_RATE = 0.01
BATCH_SIZE = 288

INPUT_SIZE = 8
HIDDEN_SIZE = 128
NUM_LAYERS = 2
OUTPUT_SIZE = 1

In [21]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, 128, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x, _ = self.lstm(x)
        x, _ = self.lstm2(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

In [22]:
model = LSTMModel(INPUT_SIZE,HIDDEN_SIZE, NUM_LAYERS, OUTPUT_SIZE).to(device)
model.load_state_dict(torch.load("models/LSTMwithzzz.model"))
model.eval()



LSTMModel(
  (lstm): LSTM(8, 128, num_layers=2, batch_first=True)
  (lstm2): LSTM(128, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [9]:
y_pred = []
y_target = []
y_prob = []
model.eval()

with torch.no_grad():
     for i, (X_train, y_valid) in enumerate(valid_loader):
        output = model(X_train).cpu()
        y_pred_p = output
        y_pred_tag = (output>0.5).int()
        y_prob.append(y_pred_p.detach().numpy())
        y_pred.append(y_pred_tag.detach().numpy())
        y_target.append(y_valid.cpu().detach().numpy())

y_pred = [a.squeeze().tolist() for a in y_pred]
y_pred = list(itertools.chain.from_iterable(y_pred))
y_target = [a.squeeze().tolist() for a in y_target]
y_target = list(itertools.chain.from_iterable(y_target))
y_prob = [a.squeeze().tolist() for a in y_prob]
y_prob = list(itertools.chain.from_iterable(y_prob))

from sklearn.metrics import zero_one_loss

zero_one_loss(y_target, y_pred)

0.00988997855812157

In [10]:
from metric import score

test_info = multi_series[2133675:2844900]

def moving_average_filter(lst, window_size):
    # Applying a simple moving average filter
    smoothed = []
    for i in range(len(lst)):
        start = max(i - window_size // 2, 0)
        end = min(i + window_size // 2 + 1, len(lst))
        smoothed.append(round(sum(lst[start:end]) / (end - start)))
    return smoothed

y_pred = moving_average_filter(y_pred, int(2*60*60/5))

def find_change_points(lst):
    change_points = []
    change_points_events = []
    for i in range(1, len(lst)):
        if lst[i] != lst[i-1]:
            change_points.append(i)
            if lst[i] == 0:
                change_points_events.append("onset")
            else:
                change_points_events.append("wakeup")
    return change_points, change_points_events

# predicted_event = return_event(test_info, y_pred, y_prob)
index_points, event = find_change_points(y_pred)
# print(event)
detected_event = test_info.iloc[index_points][["series_id", "step", "timestamp"]]


confident_score = [] #since use change point detection by hard switch, we confirm the score to be 1

for i in range(len(event)):
    confident_score.append(1)


detected_event["event"] = event
detected_event["score"] = confident_score

In [11]:
series_id_column_name = "series_id"
time_column_name = "step"
event_column_name = "event"
score_column_name = "score"
valid_event = pd.read_csv("zzz_validevent.csv")
detected_event

,series_id,step,timestamp,event,score
2139562,18b61dd5aae8,254422,2018-01-06 14:51:50+00:00,wakeup,1
2148680,18b61dd5aae8,263540,2018-01-07 03:31:40+00:00,onset,1
2154779,18b61dd5aae8,269639,2018-01-07 11:59:55+00:00,wakeup,1
2167959,18b61dd5aae8,282819,2018-01-08 06:18:15+00:00,onset,1
2174045,18b61dd5aae8,288905,2018-01-08 14:45:25+00:00,wakeup,1
...,...,...,...,...,...
2791551,29c75c018220,410871,2018-04-28 11:09:15+00:00,wakeup,1
2803198,29c75c018220,422518,2018-04-29 03:19:50+00:00,onset,1
2809058,29c75c018220,428378,2018-04-29 11:28:10+00:00,wakeup,1
2820960,29c75c018220,440280,2018-04-30 04:00:00+00:00,onset,1


In [12]:
valid_event

,series_id,night,event,step,timestamp
0,18b61dd5aae8,15,wakeup,254292,2018-01-06T09:41:00-0500
1,18b61dd5aae8,16,onset,263424,2018-01-06T22:22:00-0500
2,18b61dd5aae8,16,wakeup,269856,2018-01-07T07:18:00-0500
3,18b61dd5aae8,17,onset,282864,2018-01-08T01:22:00-0500
4,18b61dd5aae8,17,wakeup,288924,2018-01-08T09:47:00-0500
...,...,...,...,...,...
76,29c75c018220,24,wakeup,410856,2018-04-28T07:08:00-0400
77,29c75c018220,25,onset,422532,2018-04-28T23:21:00-0400
78,29c75c018220,25,wakeup,428340,2018-04-29T07:25:00-0400
79,29c75c018220,26,onset,440256,2018-04-29T23:58:00-0400


In [14]:
tolerances = {
    'onset': [12, 36, 60, 90, 120, 150, 180, 240, 300, 360, 720], 
    'wakeup': [12, 36, 60, 90, 120, 150, 180, 240, 300, 360, 720]
}


print(score
(
        valid_event,
        detected_event,
        tolerances,
        series_id_column_name,
        time_column_name,
        event_column_name,
        score_column_name
        # use_scoring_intervals=True
))


0.7208340713049592
